In [ ]:
from cltk import NLP
from cltk.data.fetch import FetchCorpus

In [ ]:
corpus_downloader = FetchCorpus(language="grc")
corpus_downloader.import_corpus("grc_text_tesserae")
cltk_nlp = NLP(language='grc')

In [ ]:
import os
import re
from tqdm.notebook import tqdm
from collections import defaultdict as dd

### Load Corpa

In [ ]:
DATA_DIR = '/home/zach/cltk_data/grc/text/grc_text_tesserae/texts/'
files = os.listdir(DATA_DIR)

docs = dd(list)
keys = []

for file in tqdm(files):
    key = os.path.splitext(file)[0]
    keys.append(key)
    filepath = '{}/{}'.format(DATA_DIR, file)
    text = re.sub('<[^<]+>', "", open(filepath, encoding="utf8").read())
    text = re.sub('\n', "", text)
    docs[key] = text

### Annotate Corpa

In [ ]:
data = dd(list)
for key in tqdm(keys[:1]):
    data[key] = cltk_nlp.analyze(text=docs[key])

In [ ]:
import os import re from cltk import NLP from tqdm.notebook import tqdm

DATA_DIR = '/home/zach/cltk_data/grc/text/grc_text_tesserae/texts/' files = os.listdir(DATA_DIR)

data = {}

for file in tqdm(files):
    filepath = '{}/{}'.format(DATA_DIR, file)
    text = re.sub('<[^<]+>', "", open(filepath, encoding="utf8").read())
    text = re.sub('\n', "", text)
    data[file] = text

cltk_doc = cltk_nlp.analyze(text=text[1])

In [ ]:
from pystlm.stlm import STLM
from pystlm.suffixtree import SuffixTree
from pystlm.sequence import Sequence

trie = SuffixTree()

doc = data[keys[0]]

for w in doc.tokens:
for w in cltk_doc.tokens:
    trie.add(w)

trie.update_all_counts()
stlm = STLM(trie)
seq = Sequence()
for t in doc.tokens[:3]: 
for t in cltk_doc.tokens[:1]: 
    seq.push_back(t)

stlm.prob(seq)